In [ ]:
#1-person/human, 0-spoof

In [1]:
import time
import glob
from IPython.display import Audio
import logging
from concurrent.futures import ThreadPoolExecutor
from joblib import Parallel, delayed
import time
import logging
from IPython.display import Audio
from tqdm import tqdm
import subprocess
import pandas as pd

from IDRnD.utils import *
from IDRnD.augmentations import *
from IDRnD.dataset import *
from IDRnD.resnet import resnet50
from IDRnD.focalloss import FocalLoss
import numpy as np

import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR, CyclicLR, ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

from sklearn.model_selection import StratifiedKFold, train_test_split

%reload_ext autoreload
%autoreload 2
%matplotlib inline

seed_everything(0)
logging.basicConfig(level=logging.DEBUG, filename="logs/logs.log",
                    filemode="w+")

In [2]:
train_transform = transforms.Compose([
    #RandomParameter(VTLP, [[0.8, 1.2]], p=0.5),
    #MinMaxChunkScaler(),
    #Normalize(),
    #RandomParameter(RandomNoise, [[0.01, 0.1]]),
    #RandomParameter(Shift, [[2000, 32000]]),
    #RandomParameter(TimeStretch, [[0.75, 1.3]]),
    #RandomParameter(PitchShift, [[-8, 8]]),
    #RandomParameter(Distortion, [[-1, -0.3], [.3, 1.]]),
    ToMellSpec(n_mels=128),
    #GetMFCC(),
    #PadOrClip(320),
    #Normalize(),
    #ToTensor(),
    #transforms.ToTensor(),
])

In [5]:
old_spoof1 = pd.read_csv("/data_spoof/CM_protocol/cm_train.trn", sep = ' ', names=["folder", "filename", "wat", "class"])
old_spoof2 = pd.read_csv("/data_spoof/CM_protocol/cm_evaluation.ndx", sep = ' ', names=["folder", "filename", "wat", "class"])
old_spoof3 = pd.read_csv("/data_spoof/CM_protocol/cm_develop.ndx", sep = ' ', names=["folder", "filename", "wat", "class"])
old_spoof = pd.concat([old_spoof1, old_spoof2, old_spoof3])
old_spoof.reset_index(drop=True, inplace=True)

In [3]:
base_dir = "/data_spoof/wav/"
pathes = np.array(old_spoof[["folder", "filename"]].apply(lambda x:os.path.join(base_dir, *x)+".wav", axis=1))
maping = {"spoof":0, "human":1}
classes = np.array(old_spoof["class"].apply(lambda x: maping[x]))

old_spoof_dataset = BaseDatasetScipy(pathes, classes, train_transform)

NameError: name 'old_spoof' is not defined

In [4]:
base_path = "/data_common_voices/en/clips_wavs/"
common_voice_files = os.listdir(base_path)
common = [os.path.join(base_path, file) for file in common_voice_files]
common_dataset = BaseDatasetScipy(common, np.ones_like(common, dtype=np.int16), train_transform)

In [5]:
k = os.listdir("/data_common_voices/en/clips")

In [6]:
len(common_dataset)

914323

In [7]:
len(k)

896823

In [27]:
Parallel(n_jobs=16, verbose=5, backend="multiprocessing")(delayed(apu.convert_to_wav)(file) for file in common_voice_files)

[Parallel(n_jobs=200)]: Using backend ThreadingBackend with 200 concurrent workers.
[Parallel(n_jobs=200)]: Done  50 tasks      | elapsed:    5.8s
[Parallel(n_jobs=200)]: Done 248 tasks      | elapsed:    7.8s
[Parallel(n_jobs=200)]: Done 482 tasks      | elapsed:   11.7s
[Parallel(n_jobs=200)]: Done 752 tasks      | elapsed:   14.9s
[Parallel(n_jobs=200)]: Done 1058 tasks      | elapsed:   19.0s
[Parallel(n_jobs=200)]: Done 1400 tasks      | elapsed:   23.6s
[Parallel(n_jobs=200)]: Done 1778 tasks      | elapsed:   28.4s
[Parallel(n_jobs=200)]: Done 2192 tasks      | elapsed:   34.0s
[Parallel(n_jobs=200)]: Done 2642 tasks      | elapsed:   34.8s


KeyboardInterrupt: 

In [9]:
def move_file(file):
    new_name = os.path.join(base_path, file.replace('.mp3.mp3', ".wav"))
    old_name = os.path.join(base_path, file)
    os.rename(old_name, new_name)

In [11]:
print(len(common_dataset))
Parallel(n_jobs=16, verbose=5)(delayed(move_file)(file) for file in common_voice_files)

914323


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  42 tasks      | elapsed:    0.6s
[Parallel(n_jobs=16)]: Done 1404 tasks      | elapsed:    1.3s


KeyboardInterrupt: 

In [10]:
def precalculate_and_save_one_epoch(folder, run, dataset):
    for sample in tqdm(range(len(dataset))):
        k = dataset[sample][0]
        name = f"{run}_{sample}"
        path = os.path.join(folder, name)
        np.save(path, k)

In [11]:
def save_sample(sample, folder):
    try:
        k = common_dataset[sample][0]#TODO
        name = common_dataset.X[sample].split("/")[-1].split(".")[0]
        #name = f"{run}_{sample}"
        path = os.path.join(folder, name)
        np.save(path, k)
    except Exception as exp:
        print(exp)

In [12]:
def precalculate_and_save_one_epoch_parallel(run, folder, dataset):
    before = time.time()
    with ThreadPoolExecutor(max_workers=16) as executor:
        for sample in range(len(dataset)):
            future = executor.submit(save_sample, sample, dataset, run, folder)
    
    print(time.time() - before)

In [13]:
def precalculate_and_save_one_epoch_joblib(folder, dataset):
    Parallel(n_jobs=16, verbose=5, backend="multiprocessing")(delayed(save_sample)(sample, folder)
                        for sample in range(len(dataset)))

!rm -rf ../data/files/raw_mels
!rm -rf ../data/files/raw_mels_val

In [ ]:
for num in range(0, 1):
    precalculate_and_save_one_epoch_joblib("../data/files/raw_mels", common_dataset)
    #precalculate_and_save_one_epoch_joblib(num, "../data/files/raw_mels_val", val_dataset)

[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done 128 tasks      | elapsed:    0.4s
[Parallel(n_jobs=16)]: Done 1208 tasks      | elapsed:    2.1s
[Parallel(n_jobs=16)]: Done 2720 tasks      | elapsed:    4.8s
[Parallel(n_jobs=16)]: Done 4664 tasks      | elapsed:    7.8s
[Parallel(n_jobs=16)]: Done 7040 tasks      | elapsed:   11.8s
[Parallel(n_jobs=16)]: Done 9848 tasks      | elapsed:   16.2s
[Parallel(n_jobs=16)]: Done 13088 tasks      | elapsed:   21.6s


There aren't any elements to reflect in axis 0 of `array`


[Parallel(n_jobs=16)]: Done 16682 tasks      | elapsed:   32.9s
[Parallel(n_jobs=16)]: Done 17762 tasks      | elapsed:   43.6s
[Parallel(n_jobs=16)]: Done 18896 tasks      | elapsed:   55.1s
[Parallel(n_jobs=16)]: Done 20138 tasks      | elapsed:  1.1min
[Parallel(n_jobs=16)]: Done 21488 tasks      | elapsed:  1.4min
[Parallel(n_jobs=16)]: Done 22946 tasks      | elapsed:  1.6min
[Parallel(n_jobs=16)]: Done 24512 tasks      | elapsed:  1.9min
[Parallel(n_jobs=16)]: Done 26186 tasks      | elapsed:  2.2min
[Parallel(n_jobs=16)]: Done 27968 tasks      | elapsed:  2.5min
[Parallel(n_jobs=16)]: Done 29858 tasks      | elapsed:  2.8min
[Parallel(n_jobs=16)]: Done 31856 tasks      | elapsed:  3.2min
[Parallel(n_jobs=16)]: Done 33962 tasks      | elapsed:  3.5min
[Parallel(n_jobs=16)]: Done 36176 tasks      | elapsed:  3.9min
[Parallel(n_jobs=16)]: Done 37794 tasks      | elapsed:  4.2min
[Parallel(n_jobs=16)]: Done 38604 tasks      | elapsed:  4.3min
[Parallel(n_jobs=16)]: Done 39450 tasks 

There aren't any elements to reflect in axis 0 of `array`


[Parallel(n_jobs=16)]: Done 212700 tasks      | elapsed: 35.1min
[Parallel(n_jobs=16)]: Done 216354 tasks      | elapsed: 35.8min
[Parallel(n_jobs=16)]: Done 220044 tasks      | elapsed: 36.4min
[Parallel(n_jobs=16)]: Done 223770 tasks      | elapsed: 37.1min
[Parallel(n_jobs=16)]: Done 227532 tasks      | elapsed: 37.8min
[Parallel(n_jobs=16)]: Done 231330 tasks      | elapsed: 38.4min
[Parallel(n_jobs=16)]: Done 235164 tasks      | elapsed: 39.1min
[Parallel(n_jobs=16)]: Done 239034 tasks      | elapsed: 39.8min
[Parallel(n_jobs=16)]: Done 242940 tasks      | elapsed: 40.5min
[Parallel(n_jobs=16)]: Done 246882 tasks      | elapsed: 41.2min
[Parallel(n_jobs=16)]: Done 250860 tasks      | elapsed: 41.9min
[Parallel(n_jobs=16)]: Done 254874 tasks      | elapsed: 42.6min
[Parallel(n_jobs=16)]: Done 258924 tasks      | elapsed: 43.3min
[Parallel(n_jobs=16)]: Done 263010 tasks      | elapsed: 44.0min
[Parallel(n_jobs=16)]: Done 267132 tasks      | elapsed: 44.8min
[Parallel(n_jobs=16)]: Do

...

[Parallel(n_jobs=16)]: Done 536241 tasks      | elapsed: 92.7min
[Parallel(n_jobs=16)]: Done 542271 tasks      | elapsed: 93.8min
[Parallel(n_jobs=16)]: Done 548337 tasks      | elapsed: 94.8min
[Parallel(n_jobs=16)]: Done 554439 tasks      | elapsed: 95.9min
[Parallel(n_jobs=16)]: Done 560577 tasks      | elapsed: 97.0min
[Parallel(n_jobs=16)]: Done 566751 tasks      | elapsed: 98.1min
[Parallel(n_jobs=16)]: Done 572961 tasks      | elapsed: 99.2min
[Parallel(n_jobs=16)]: Done 579207 tasks      | elapsed: 100.3min
[Parallel(n_jobs=16)]: Done 585489 tasks      | elapsed: 101.4min
[Parallel(n_jobs=16)]: Done 591807 tasks      | elapsed: 102.6min
[Parallel(n_jobs=16)]: Done 598161 tasks      | elapsed: 103.7min
[Parallel(n_jobs=16)]: Done 604551 tasks      | elapsed: 104.8min
[Parallel(n_jobs=16)]: Done 610977 tasks      | elapsed: 106.0min
[Parallel(n_jobs=16)]: Done 617439 tasks      | elapsed: 107.2min
[Parallel(n_jobs=16)]: Done 623937 tasks      | elapsed: 108.7min
[Parallel(n_jobs=

There aren't any elements to reflect in axis 0 of `array`


[Parallel(n_jobs=16)]: Done 637041 tasks      | elapsed: 111.0min
[Parallel(n_jobs=16)]: Done 643647 tasks      | elapsed: 112.2min
[Parallel(n_jobs=16)]: Done 650289 tasks      | elapsed: 113.4min
[Parallel(n_jobs=16)]: Done 656967 tasks      | elapsed: 114.6min
[Parallel(n_jobs=16)]: Done 663681 tasks      | elapsed: 115.8min
[Parallel(n_jobs=16)]: Done 670431 tasks      | elapsed: 117.0min


There aren't any elements to reflect in axis 0 of `array`


[Parallel(n_jobs=16)]: Done 677217 tasks      | elapsed: 118.2min
[Parallel(n_jobs=16)]: Done 684039 tasks      | elapsed: 119.4min
[Parallel(n_jobs=16)]: Done 690897 tasks      | elapsed: 120.6min
[Parallel(n_jobs=16)]: Done 697791 tasks      | elapsed: 121.8min
[Parallel(n_jobs=16)]: Done 704721 tasks      | elapsed: 123.0min
[Parallel(n_jobs=16)]: Done 711687 tasks      | elapsed: 124.3min
[Parallel(n_jobs=16)]: Done 718689 tasks      | elapsed: 125.5min


In [15]:
len(common_dataset)

914323

In [ ]:
len(os.listdir("../data/files/raw_mels/"))